<a href="https://colab.research.google.com/github/hamza3e/keras_mlp_ionosphere/blob/master/ionosphere_mlp_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q keras

In [0]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data

In [0]:
import numpy as np
import pandas as pd

dataset = pd.read_csv("ionosphere.data", header=None)
X = dataset.values[0:350,0:34]
labels = dataset.values[0:350,34]
Y = np.zeros(labels.shape)
Y[labels=='g'] = 1
Y[labels=='b'] = 0
print(Y)

In [0]:
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
import numpy as np
# fix random seed for reproducibility
seed =7
np.random.seed(seed)

In [0]:
kf = KFold(n_splits=10, random_state=None, shuffle=True)
kf.get_n_splits(X)
cvscores = []
cvsensitivity = []
cvspecificity = []


print(kf) 

In [0]:
model = Sequential()
model.add(Dense(80, input_dim=34, activation='sigmoid'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.1, momentum=0.91, decay=0.0, nesterov=False)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])

In [0]:
for train, test in kf.split(X):

  model.fit(X[train], Y[train], epochs=10, batch_size=35, verbose=0)
	# evaluate the model

  scores = model.evaluate(X[test], Y[test], verbose=0)
  
  y_test = Y[test]
  y_pred = model.predict_classes(X[test])
  tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
  specificity = tn / (tn+fp)
  sensitivity = tp / (tp+fn)
  print(" specificity: %.2f%% \n sensitivity: %.2f%%" % (specificity*100, sensitivity*100))
  
  cvsensitivity.append(sensitivity * 100)
  cvspecificity.append(specificity * 100)
  
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

  cvscores.append(scores[1] * 100)
  
  
print("Mean of acc: %.2f%% (+/- %.2f%%) " % (np.mean(cvscores), np.std(cvscores)))

print("Mean of sen: %.2f%% (+/- %.2f%%) " % (np.mean(cvsensitivity), np.std(cvsensitivity)))
print("Mean of spe: %.2f%% (+/- %.2f%%) " % (np.mean(cvsensitivity), np.std(cvspecificity)))

from keras.backend import clear_session
clear_session()
cvscores.clear()
cvsensitivity.clear()
cvspecificity.clear()

In [0]:

from keras.backend import clear_session
clear_session()